<a href="https://colab.research.google.com/github/cyyoon0311/Machine-learning/blob/main/Homework4_2021111939_%EC%9C%A4%EC%B0%AC%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report


In [7]:
# CSV 파일 읽기
df = pd.read_csv("/content/sample_data/audit_data.csv.xls", encoding='latin1')

# 결측치 제거
df = df.dropna()

# 불필요한 열 제거
df = df.drop(columns=['LOCATION_ID'])

# X, y 분리
X = df.drop(columns=['Risk'])
y = df['Risk']

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 75:25 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)


In [8]:
# 모델 정의
knn = KNeighborsClassifier()
svm = SVC(probability=True)
tree = DecisionTreeClassifier(random_state=42)

# 학습
knn.fit(X_train, y_train)
svm.fit(X_train, y_train)
tree.fit(X_train, y_train)

# 예측
knn_pred = knn.predict(X_test)
svm_pred = svm.predict(X_test)
tree_pred = tree.predict(X_test)

# 평가
print("KNN 결과:\n", classification_report(y_test, knn_pred))
print("SVM 결과:\n", classification_report(y_test, svm_pred))
print("Decision Tree 결과:\n", classification_report(y_test, tree_pred))


KNN 결과:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       118
           1       1.00      0.96      0.98        76

    accuracy                           0.98       194
   macro avg       0.99      0.98      0.98       194
weighted avg       0.98      0.98      0.98       194

SVM 결과:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       118
           1       0.99      0.97      0.98        76

    accuracy                           0.98       194
   macro avg       0.98      0.98      0.98       194
weighted avg       0.98      0.98      0.98       194

Decision Tree 결과:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       118
           1       1.00      1.00      1.00        76

    accuracy                           1.00       194
   macro avg       1.00      1.00      1.00       194
weighted avg       1.00      1.00    

In [9]:
# 소프트 보팅 앙상블 (확률 평균 기반)
voting_clf = VotingClassifier(
    estimators=[('knn', knn), ('svm', svm), ('tree', tree)],
    voting='soft',
    weights=[1, 2, 1]
)

# 학습 및 예측
voting_clf.fit(X_train, y_train)
voting_pred = voting_clf.predict(X_test)

# 평가
print("VotingClassifier 결과:\n", classification_report(y_test, voting_pred))


VotingClassifier 결과:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       118
           1       1.00      0.99      0.99        76

    accuracy                           0.99       194
   macro avg       1.00      0.99      0.99       194
weighted avg       0.99      0.99      0.99       194



In [ ]:
# KNN 모델은 precision, recall, f1-score가 모두 0.98로 비교적 우수한 성능을 보였으나, Decision Tree 및 앙상블 모델에 비해 소폭 낮은 결과를 보였다.
# SVM 모델은 KNN과 유사한 수준의 성능을 보였으며, 클래스 1(위험)의 recall이 0.97로 약간 낮게 측정되었다.
# Decision Tree 모델은 테스트셋 기준으로 모든 성능 지표가 1.00으로 나타났으며, 이는 오버피팅 가능성도 존재하지만 현재 데이터 기준으로는 최고의 성능을 보였다.
# VotingClassifier 앙상블 모델은 precision과 recall이 각각 0.99와 1.00 수준으로, 전체적으로 단일 모델보다 더 안정적인 성능을 나타냈다. 특히, 클래스 1의 recall이 0.99로 높게 유지되면서 precision도 높아, 불균형 상황에서도 강건한 모델로 평가된다.
# 결론적으로, 앙상블 모델은 모든 단일 모델보다 최소 하나 이상의 지표에서 높은 성능을 기록하였으며, 특히 데이터 불균형이나 다양한 특성을 안정적으로 반영할 수 있다는 점에서 최종적으로 가장 우수한 모델이라고 판단된다.